#### Installing Packages

<i>%pip install tensorflow <br>
%pip install keras<br>
%pip install matplotlib<br>
%pip install pillow<br>
%pip install tqdm<br>
%pip install -U scikit-learn</i>

In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from PIL import Image, ImageFile
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [2]:
IMAGE_SIZE = [300, 400]
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in vgg.layers:
    layer.trainable = False

In [3]:
x = Flatten()(vgg.output)
preds = Dense(2, activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=preds)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 300, 400, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 300, 400, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 300, 400, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 150, 200, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 150, 200, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 150, 200, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 75, 100, 128)      0     

In [4]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=['accuracy'],
)

In [5]:
ImageFile.LOAD_TRUNCATED_IMAGES = True
CLASSIFICATION_DATA_PATH = "../../my-datasets/FloodNet Challenge @ EARTHVISION 2021 - Track 1/Train/Classification"
CATEGORIES = ["Flooded", "Non-Flooded"]
data_list = []

for category in CATEGORIES:
    path = os.path.join(CLASSIFICATION_DATA_PATH, category)
    class_num = CATEGORIES.index(category)
    for img_path in tqdm(os.listdir(path)):
        try:
            img = Image.open(os.path.join(path, img_path))
            img = np.array(img.resize((400, 300)))
            data_list.append([img, class_num])
        except Exception as e:
            print(e)
print(f"Total number of samples is {len(data_list)}")

100%|██████████| 347/347 [01:16<00:00,  4.55it/s]

Total number of samples is 480


In [6]:
import random
random.shuffle(data_list)

X = []
y = []

for image, label in data_list:
    X.append(image)
    y.append(label)

X = np.array(X).reshape(-1, 300, 400, 3)
y = np.array(y)

In [7]:
'''
np.savez_compressed("data/X_images", X, allow_pickle=True)
np.savez_compressed("data/y_labels", y, allow_pickle=True)
X = np.load("X_images.npy")
y = np.load("y_labels.npy")
'''

'\nnp.savez_compressed("data/X_images", X, allow_pickle=True)\nnp.savez_compressed("data/y_labels", y, allow_pickle=True)\nX = np.load("X_images.npy")\ny = np.load("y_labels.npy")\n'

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=29)

print(f"Number of samples in train set: {len(X_train)}")
print(f"Number of samples in test set: {len(X_test)}")

model.fit(
    x=X_train,
    y=y_train,
    batch_size=16,
    epochs=3,
    verbose=1,
) 

Number of samples in train set: 360
Number of samples in test set: 120
Epoch 1/3
23/23 [==============================] - 15s 311ms/step - loss: 4.5791 - accuracy: 0.8750
Epoch 2/3
23/23 [==============================] - 3s 141ms/step - loss: 0.4940 - accuracy: 0.9750
Epoch 3/3
23/23 [==============================] - 3s 140ms/step - loss: 8.4826e-04 - accuracy: 1.0000


In [8]:
#model.evaluate(x=X, y=y, batch_size=16)
preds = model.predict(x=X_test)
preds_ = np.argmax(preds, axis=1)

total = 0
correct = 0

for pred, truth in zip(preds_, y_test):
    if(pred == truth):
        correct += 1
    total += 1
    
print(f"Accuracy: {(correct/total)*100}%")

Accuracy: 96.66666666666667%


In [10]:
#model.save("vgg_classification_model")
#my_model = keras.models.load_model("vgg_classification_model")'''

2022-01-17 14:49:51.805240: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: vgg_classification_model/assets


In [ ]:
my_model = keras.models.load_model("vgg_classification_model")

prediction = my_model.predict(x=